In [1]:
import pandas as pd
import pyam
import aneris
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
gem_e3_path = '../../../data/SOD/model_results/raw/GEM_E3_v2.xlsx'

In [3]:
df_unclean = pd.read_excel(gem_e3_path)
df_unclean['Scenario'] = df_unclean['Scenario'].replace({'DECARB_LIFE': 'LIFE-TP-v2',
                                                        'DECARB_TECH': 'TECH-TP-v2',
                                                        'REF_V2':'REF-v2',
                                                  })

gem_vars = [
    'Employment|Agriculture',
    'Employment|Industry',
    'Employment|Service',
    'Investment|Energy Supply',
    'Unemployment|Rate',
    'Investment',
    'Import|Agriculture',
    'Import|Energy',
    'Import|Industry',
    'Export|Agriculture',
    'Export|Energy',
    'Export|Industry',
    'Policy Cost|GDP Loss',
    'GDP|MER',
    'Population'
]

df_unclean = df_unclean[df_unclean['Variable'].isin(gem_vars)]

df_unclean

,Model,Scenario,Region,Variable,Unit,2015,2020,2025,2030,2035,...,2055,2060,2065,2070,2075,2080,2085,2090,2095,2100
0,GEM-E3_V2023,TECH-TP-v2,USA,Population,million,320.878312,331.002647,340.399604,349.641876,358.690997,...,385.334853,391.494892,397.876094,404.174404,409.992638,415.197398,419.937438,424.469922,429.081664,433.853891
1,GEM-E3_V2023,TECH-TP-v2,USA,GDP|MER,billion US$2010/yr,16525.308993,17117.886092,19252.550809,21181.228893,23104.264774,...,34319.115491,38207.561189,40480.144470,42769.324497,44741.294765,46969.882983,49143.775767,51313.930337,53478.145948,55670.627796
218,GEM-E3_V2023,TECH-TP-v2,USA,Policy Cost|GDP Loss,billion US$2010/yr,0.000000,-0.000002,90.123246,289.138206,367.960304,...,547.468142,555.926798,564.775875,559.675994,553.369149,543.027659,558.150306,585.846902,633.208201,719.365980
258,GEM-E3_V2023,TECH-TP-v2,USA,Export|Agriculture,billion US$2010/yr,83.063055,107.110047,142.506502,175.783790,218.571221,...,528.308762,635.033331,730.627786,827.261365,895.920082,986.535831,1063.143430,1124.778000,1191.779906,1251.129509
259,GEM-E3_V2023,TECH-TP-v2,USA,Export|Energy,billion US$2010/yr,156.899619,147.704675,145.930945,130.557375,119.336096,...,70.452286,62.748403,58.812599,54.622179,49.707007,46.601037,44.560221,43.512188,42.685134,42.117837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30587,GEM-E3_V2023,LIFE-TP-v2,World,Employment|Service,million,1710.914047,1678.416969,1752.349690,1785.724832,1814.215949,...,1902.353426,1918.629656,1924.976206,1930.324887,1931.737052,1931.857938,1928.953842,1927.052581,1924.762897,1918.775577
30588,GEM-E3_V2023,LIFE-TP-v2,World,Import|Industry,billion US$2010/yr,13485.393329,13798.921771,16522.432415,18945.438772,21559.920689,...,36139.810812,40517.349499,44160.603012,48192.753314,53080.349294,58078.064250,63278.579627,68880.927771,74740.721470,80996.446954
30590,GEM-E3_V2023,LIFE-TP-v2,World,Investment|Energy Supply,billion US$2010/yr,1681.279861,1767.557921,1818.171769,1790.911504,1727.237094,...,1748.326886,1939.454630,1977.396240,2164.191305,1959.617653,2122.988116,1876.102282,2049.176756,2156.673502,2439.091933
30666,GEM-E3_V2023,LIFE-TP-v2,World,Unemployment|Rate,%,5.478013,4.451232,5.242275,5.760162,6.237122,...,6.372672,6.270089,6.314310,6.306686,6.208586,6.051267,5.873510,5.699815,5.523036,5.348510


In [4]:
def compute_UN_groups(df, variables, weight=None):
    df_africa = df.aggregate_region(variables,
                              region='Africa (UN-R5)',
                              subregions=[
                                    'SAF', 
                                    'ROW'
                              ],
                              weight=weight
                              )

    df_asia = df.aggregate_region(variables,
                              region='Asia and the Pacific (UN-R5)',
                              subregions=[
                                'JPN', 
                                'CHN',
                                'IND',
                                'KOR',
                                'IDN',
                                'SAR',
                                'REP',
                              ],
                              weight=weight
                              )

    df_ee = df.aggregate_region(variables,
                              region='Eastern Europe (UN-R5)',
                              subregions=[
                                'REU',
                                'RUS',
                              ],
                              weight=weight
                              )

    df_latin = df.aggregate_region(variables,
                              region='Latin America and Caribbean (UN-R5)',
                              subregions=[
                                'ARG',
                                'BRA',
                                'MEX',
                              ],
                              weight=weight
                              )

    df_we = df.aggregate_region(variables,
                              region='Western Europe and Other States (UN-R5)',
                              subregions=[
                                'EU28',
                                'USA',
                                'CAN',
                                'TUR',
                                'OCE',
                              ],
                              weight=weight
                              )
    

    df_un_groups = pyam.concat([df_africa, df_asia, df_ee, df_latin, df_we])
    return df_un_groups


def compute_3_groups_gem(df, variables, weight=None):
    df_high_income = df.aggregate_region(variables,
                              region='High-Income',
                              subregions=[
                                'USA',
                                'CAN',
                                'JPN',
                                'OCE',
                                'EU28'
                              ],
                              weight=weight
                              )

    df_middle_income = df.aggregate_region(variables,
                              region='Middle-Income',
                              subregions=[
                                'MEX',
                                'SAF',
                                'TUR',
                                'BRA',
                                'CHN',
                                'IDN',
                                'RUS',
                              ],
                              weight=weight
                              )

    df_low_income = df.aggregate_region(variables,
                              region='Low-Income',
                              subregions=[
                                'IND',
                                'ROW',
                              ],
                              weight=weight
                              )

    df_3_groups = pyam.concat([df_high_income, df_middle_income, df_low_income])
    return df_3_groups


def compute_6_groups_gem(df, variables, weight=None):
    df_high_low = df.aggregate_region(variables,
                              region='High-Income, Low Density',
                              subregions=[
                                'USA',
                                'CAN',
                                'OCE',
                              ],
                              weight=weight
                              )

    df_middle_low = df.aggregate_region(variables,
                              region='Middle-Income, Low Density',
                              subregions=[
                                'MEX',
                                'SAF',
                                'BRA',
                                'RUS',
                              ],
                              weight=weight
                              )

    # df_low_low = df.aggregate_region(variables,
    #                           region='Low-Income, Low Density',
    #                           subregions=[
                                
    #                           ],
    #                           weight=weight
    #                           )

    df_high_high = df.aggregate_region(variables,
                              region='High-Income, High Density',
                              subregions=[
                                'JPN',
                                'EU28'
                              ],
                              weight=weight
                              )

    df_middle_high = df.aggregate_region(variables,
                              region='Middle-Income, High Density',
                              subregions=[
                                'TUR',
                                'CHN',
                              ],
                              weight=weight
                              )

    # df_low_high = df.aggregate_region(variables,
    #                           region='Low-Income, High Density',
    #                           subregions=[
    #                             'IND',
    #                             'ROW',
    #                           ],
    #                           weight=weight
    #                           )

    df_6_groups = pyam.concat([df_high_low,
                              df_middle_low,
                              # df_low_low,
                              df_high_high,
                              df_middle_high,
                              # df_low_high
                              ])
    return df_6_groups

In [5]:
df_mid_clean = pyam.IamDataFrame(df_unclean)
df_mid_clean.divide("Policy Cost|GDP Loss", 
                "GDP|MER", 
                "GDP|Climate Policy Cost|Relative", 
                ignore_units='', 
                append=True
                )
df_mid_clean.rename(unit={'billion US$2010/yr':'billion USD_2010/yr'}, inplace=True)
df_mid_clean.convert_unit('', '%', inplace=True)
df_mid_clean.aggregate("Import", 
                        append=True)
df_mid_clean.aggregate("Export", 
                        append=True)


In [6]:
df_mid_clean_absolute = compute_UN_groups(df_mid_clean, df_mid_clean.filter(unit='%', keep=False).variable)
df_mid_clean_unemployement = compute_UN_groups(df_mid_clean, variables=['Unemployment|Rate'], weight='Population')
df_mid_clean_gdp = compute_UN_groups(df_mid_clean, variables=['GDP|Climate Policy Cost|Relative'], weight='GDP|MER')
df_mid_clean_world = df_mid_clean.filter(region='World')

df_clean = pyam.concat([df_mid_clean_absolute, df_mid_clean_unemployement, df_mid_clean_gdp, df_mid_clean_world])

# chapter 19 requested GDP per Capita
df_3_groups = compute_3_groups_gem(df_mid_clean, ['GDP|MER', 'Population'])
df_6_groups = compute_6_groups_gem(df_mid_clean, ['GDP|MER', 'Population'])


df_clean = pyam.concat([df_mid_clean_absolute, df_mid_clean_unemployement, df_mid_clean_gdp, df_mid_clean_world])
df_clean.timeseries()

2015  \
model        scenario   region         variable                 unit                                
GEM-E3_V2023 LIFE-TP-v2 Africa (UN-R5) Employment|Agriculture   million                181.739475   
                                       Employment|Industry      million                320.350691   
                                       Employment|Service       million                415.496487   
                                       Export                   billion USD_2010/yr   1812.307403   
                                       Export|Agriculture       billion USD_2010/yr     93.004185   
...                                                                                           ...   
             TECH-TP-v2 World          Investment               billion USD_2010/yr  18961.677285   
                                       Investment|Energy Supply billion USD_2010/yr   1681.279861   
                                       Policy Cost|GDP Loss     billion USD_2010/yr      0.000000   
                                       Population               million               7378.539257   
                                       Unemployment|Rate        %                        5.478013   

                                                                                             2020  \
model        scenario   region         variable                 unit                                
GEM-E3_V2023 LIFE-TP-v2 Africa (UN-R5) Employment|Agriculture   million                188.919031   
                                       Employment|Industry      million                304.269638   
                                       Employment|Service       million                421.908337   
                                       Export                   billion USD_2010/yr   1803.865618   
                                       Export|Agriculture       billion USD_2010/yr    108.456562   
...                                                                                           ...   
             TECH-TP-v2 World          Investment               billion USD_2010/yr  20936.878075   
                                       Investment|Energy Supply billion USD_2010/yr   1767.557923   
                                       Policy Cost|GDP Loss     billion USD_2010/yr     -0.000059   
                                       Population               million               7795.136232   
                                       Unemployment|Rate        %                        4.451231   

                                                                                             2025  \
model        scenario   region         variable                 unit                                
GEM-E3_V2023 LIFE-TP-v2 Africa (UN-R5) Employment|Agriculture   million                199.569116   
                                       Employment|Industry      million                321.369251   
                                       Employment|Service       million                450.975092   
                                       Export                   billion USD_2010/yr   2273.152077   
                                       Export|Agriculture       billion USD_2010/yr    141.106183   
...                                                                                           ...   
             TECH-TP-v2 World          Investment               billion USD_2010/yr  24927.947159   
                                       Investment|Energy Supply billion USD_2010/yr   1826.396900   
                                       Policy Cost|GDP Loss     billion USD_2010/yr    312.676497   
                                       Population               million               8188.233699   
                                       Unemployment|Rate        %                        5.245303   

                                                                                             2030  \
model        scenario   region         variable   

In [7]:
def round_to_0_1_percent(value):
    if value == 0:
        return 0
    # Determine the number of significant digits to round to
    magnitude = np.floor(np.log10(abs(value))) - 2
    rounding_factor = 10 ** magnitude
    return np.round(value / rounding_factor) * rounding_factor

In [8]:
df_to_share = df_clean.filter(variable=['Population', 'GDP|MER', "Policy Cost|GDP Loss"], keep=False).timeseries().reset_index()

# ratio_mask = df_to_share['unit'] == '%'
# # Apply rounding to the 'value' column for rows where the 'unit' column is '%'
# df_to_share.loc[ratio_mask, df_to_share.columns[5:]] = round(df_to_share.loc[ratio_mask, df_to_share.columns[5:]])

df_to_share.loc[:, df_to_share.columns[5:]] = df_to_share.loc[:, df_to_share.columns[5:]].applymap(round_to_0_1_percent)
df_to_share

C:\Users\maycocks\AppData\Local\Temp\2\ipykernel_4240\1263476680.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_to_share.loc[:, df_to_share.columns[5:]] = df_to_share.loc[:, df_to_share.columns[5:]].applymap(round_to_0_1_percent)


,model,scenario,region,variable,unit,2015,2020,2025,2030,2035,...,2055,2060,2065,2070,2075,2080,2085,2090,2095,2100
0,GEM-E3_V2023,LIFE-TP-v2,Africa (UN-R5),Employment|Agriculture,million,182.00,189.00,200.00,213.0,223.00,...,290.0,306.00,326.0,345.00,361.00,377.00,391.00,403.00,412.00,420.00
1,GEM-E3_V2023,LIFE-TP-v2,Africa (UN-R5),Employment|Industry,million,320.00,304.00,321.00,340.0,360.00,...,435.0,454.00,470.0,487.00,506.00,521.00,535.00,549.00,562.00,574.00
2,GEM-E3_V2023,LIFE-TP-v2,Africa (UN-R5),Employment|Service,million,415.00,422.00,451.00,479.0,507.00,...,613.0,635.00,653.0,668.00,681.00,692.00,700.00,707.00,713.00,716.00
3,GEM-E3_V2023,LIFE-TP-v2,Africa (UN-R5),Export,billion USD_2010/yr,1810.00,1800.00,2270.00,2840.0,3640.00,...,8600.0,10200.00,11500.0,13100.00,14700.00,16600.00,18500.00,20600.00,22900.00,25300.00
4,GEM-E3_V2023,LIFE-TP-v2,Africa (UN-R5),Export|Agriculture,billion USD_2010/yr,93.00,108.00,141.00,186.0,248.00,...,654.0,774.00,893.0,1020.00,1120.00,1250.00,1400.00,1560.00,1730.00,1890.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,GEM-E3_V2023,TECH-TP-v2,World,Import|Energy,billion USD_2010/yr,2440.00,2180.00,2260.00,2090.0,1870.00,...,947.0,880.00,805.0,735.00,697.00,682.00,678.00,686.00,688.00,693.00
266,GEM-E3_V2023,TECH-TP-v2,World,Import|Industry,billion USD_2010/yr,13500.00,13800.00,16600.00,19000.0,21600.00,...,36400.0,40900.00,44700.0,48900.00,53900.00,59000.00,64400.00,70200.00,76300.00,82800.00
267,GEM-E3_V2023,TECH-TP-v2,World,Investment,billion USD_2010/yr,19000.00,20900.00,24900.00,28300.0,31700.00,...,49000.0,55400.00,60800.0,66900.00,73700.00,81400.00,89800.00,99400.00,110000.00,122000.00
268,GEM-E3_V2023,TECH-TP-v2,World,Investment|Energy Supply,billion USD_2010/yr,1680.00,1770.00,1830.00,1800.0,1730.00,...,1730.0,1950.00,2000.0,2190.00,1990.00,2160.00,1910.00,2080.00,2200.00,2480.00


In [9]:
df_to_share.unit.unique()

array(['million', 'billion USD_2010/yr', '%'], dtype=object)

In [10]:
df_to_share.to_excel('../../../data/SOD/model_results/to_share/GEM_E3_to_share.xlsx', index=False)